In [ ]:
from ultralytics import YOLO
import cv2
import random






In [ ]:
video_path = '/home/roni/dev_ws/yolo_tut/datasets/le-sserafim-reuserapim-crazy-official-mv_video_720p_original.mp4'

cap = cv2.VideoCapture(video_path)

In [8]:
model_path = "/home/roni/dev_ws/yolo_tut/src/runs/detect/train/weights/best.pt"
model = YOLO(model_path)

class_names = model.names

colors = {cls_id: [random.randint(0, 255) for _ in range(3)] for cls_id in class_names}

if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

In [9]:
while True:
    ret, frame = cap.read()
    
    frame = cv2.resize(frame, (1280, 720))
    
    if not ret:
        break\

    results = model(frame)

    boxes = results[0].boxes

    for box in boxes:
        
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        conf = float(box.conf[0])
        cls_id = int(box.cls[0])

        label = f"{class_names[cls_id]} {conf:.2f}"

        color = colors(cls_id)

        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(frame, label, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    cv2.imshow("Inference", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

0: 384x640 (no detections), 10.4ms
Speed: 3.4ms preprocess, 10.4ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 5.7ms
Speed: 1.6ms preprocess, 5.7ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 6.7ms
Speed: 2.8ms preprocess, 6.7ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 6.6ms
Speed: 2.0ms preprocess, 6.6ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 6.6ms
Speed: 1.6ms preprocess, 6.6ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 6.3ms
Speed: 2.8ms preprocess, 6.3ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 6.2ms
Speed: 2.2ms preprocess, 6.2ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 7.6ms
Speed: 3.1ms preprocess, 7.6ms inference, 1.2ms

error: OpenCV(4.12.0) /io/opencv/modules/imgproc/src/resize.cpp:4208: error: (-215:Assertion failed) !ssize.empty() in function 'resize'


In [4]:
import os
import json
import shutil
import random
import yaml
from pathlib import Path
from ultralytics import YOLO

# 1. Configuration
SOURCE_DIR = '/home/roni/dev_ws/yolo_tut/datasets/le-sserafim-reuserapim-crazy-official-mv_video_720p_original'
DATASET_NAME = 'lesserafim'
BASE_DIR = f'/home/roni/dev_ws/yolo_tut/datasets/{DATASET_NAME}'
IMAGES_DIR = os.path.join(BASE_DIR, 'images')
LABELS_DIR = os.path.join(BASE_DIR, 'labels')

# Create directory structure
for split in ['train', 'val']:
    os.makedirs(os.path.join(IMAGES_DIR, split), exist_ok=True)
    os.makedirs(os.path.join(LABELS_DIR, split), exist_ok=True)

# 2. Convert LabelMe JSON to YOLO txt
classes = set()
data_pairs = []

# First pass to find all pairs and classes
for filename in os.listdir(SOURCE_DIR):
    if filename.endswith('.json'):
        json_path = os.path.join(SOURCE_DIR, filename)
        image_name = filename.replace('.json', '.jpg')
        image_path = os.path.join(SOURCE_DIR, image_name)
        
        if os.path.exists(image_path):
            data_pairs.append((json_path, image_path))
            # Extract classes
            with open(json_path, 'r') as f:
                data = json.load(f)
                for shape in data['shapes']:
                    classes.add(shape['label'])

class_list = sorted(list(classes))
class_to_id = {cls: i for i, cls in enumerate(class_list)}
print(f"Classes found: {class_list}")

# Shuffle and split
random.shuffle(data_pairs)
split_idx = int(len(data_pairs) * 0.8)
train_pairs = data_pairs[:split_idx]
val_pairs = data_pairs[split_idx:]

def process_pair(pair, split):
    json_file, img_file = pair
    filename = os.path.basename(img_file)
    basename = os.path.splitext(filename)[0]
    
    # Copy image
    dst_img_path = os.path.join(IMAGES_DIR, split, filename)
    shutil.copy(img_file, dst_img_path)
    
    # Process label
    with open(json_file, 'r') as f:
        data = json.load(f)
        
    img_h = data['imageHeight']
    img_w = data['imageWidth']
    
    yolo_lines = []
    for shape in data['shapes']:
        label = shape['label']
        cls_id = class_to_id[label]
        points = shape['points']
        
        # Calculate bounding box from points
        x_coords = [p[0] for p in points]
        y_coords = [p[1] for p in points]
        
        x_min = min(x_coords)
        x_max = max(x_coords)
        y_min = min(y_coords)
        y_max = max(y_coords)
        
        # Convert to center x, center y, width, height (normalized)
        dw = 1.0 / img_w
        dh = 1.0 / img_h
        
        w = x_max - x_min
        h = y_max - y_min
        x_center = x_min + w / 2.0
        y_center = y_min + h / 2.0
        
        w *= dw
        h *= dh
        x_center *= dw
        y_center *= dh
        
        yolo_lines.append(f"{cls_id} {x_center} {y_center} {w} {h}")
        
    # Write label file
    dst_label_path = os.path.join(LABELS_DIR, split, f"{basename}.txt")
    with open(dst_label_path, 'w') as f:
        f.write('\n'.join(yolo_lines))

print("Processing train set...")
for pair in train_pairs:
    process_pair(pair, 'train')

print("Processing val set...")
for pair in val_pairs:
    process_pair(pair, 'val')

# 3. Create dataset.yaml
yaml_content = {
    'path': BASE_DIR,
    'train': 'images/train',
    'val': 'images/val',
    'names': {i: name for i, name in enumerate(class_list)}
}

yaml_path = os.path.join(BASE_DIR, 'dataset.yaml')
with open(yaml_path, 'w') as f:
    yaml.dump(yaml_content, f)

print(f"Dataset preparation complete. YAML saved to {yaml_path}")

# 4. Train YOLO
model = YOLO("yolov8n.pt") 
results = model.train(data=yaml_path, epochs=10, imgsz=640)


Classes found: ['eye', 'man', 'woman']
Processing train set...
Processing val set...
Dataset preparation complete. YAML saved to /home/roni/dev_ws/yolo_tut/datasets/lesserafim/dataset.yaml
Ultralytics 8.3.235 🚀 Python-3.12.3 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 2060, 5739MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/roni/dev_ws/yolo_tut/datasets/lesserafim/dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300

### segmentation

In [10]:
from ultralytics import YOLO


In [12]:
model = YOLO('yolov8n-seg.pt')
results = model(source='https://ultralytics.com/images/bus.jpg', save=True)

results



image 1/1 /home/roni/dev_ws/yolo_tut/src/bus.jpg: 640x480 4 persons, 1 bus, 1 skateboard, 45.4ms
Speed: 1.7ms preprocess, 45.4ms inference, 6.7ms postprocess per image at shape (1, 3, 640, 480)
Results saved to /home/roni/dev_ws/yolo_tut/src/runs/segment/predict


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: ultralytics.engine.results.Masks object
 names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 

In [13]:
model = YOLO('yolov8n-seg.pt')
results = model(source='https://ultralytics.com/images/zidane.jpg', save=True)
results


image 1/1 /home/roni/dev_ws/yolo_tut/src/zidane.jpg: 384x640 2 persons, 1 tie, 15.9ms
Speed: 2.3ms preprocess, 15.9ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)
Results saved to /home/roni/dev_ws/yolo_tut/src/runs/segment/predict2


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: ultralytics.engine.results.Masks object
 names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 

In [ ]:
model = YOLO('yolov8n-seg.pt')

results = model.train(data = 'coco8-seg.yaml', epochs = 100, imgsz = 640, save=True)


Ultralytics 8.3.235 🚀 Python-3.12.3 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 2060, 5739MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=coco8-seg.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, p

: 

In [1]:
from ultralytics import YOLO


In [2]:
model = YOLO('yolov8n-pose.pt')

results = model.train(data = "coco8-pose.yaml", epochs = 100, imgsz = 640)

Ultralytics 8.3.235 🚀 Python-3.12.3 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 2060, 5739MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=coco8-pose.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-pose.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, 

In [4]:
source_path = "/home/roni/dev_ws/yolo_tut/datasets/people_walk.mp4"


model = YOLO("yolov8n-pose.pt")
model(source = source_path, show = True, imgsz = 640)


WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/341) /home/roni/dev_ws/yolo_tut/datasets/people_walk.mp4: 384x640 6 persons, 39.1ms
video 1/1 (frame 2/341) /home/roni/dev_ws/yolo_tut/datasets/people_walk.mp4: 384x640 8 persons, 6.7ms
video 1/1 (frame 3/341) /home/roni/dev_ws/yolo_tut/datasets/people_walk.mp4: 384x640 7 persons, 6.3ms
video 1/1 (frame 4/341) /home/roni/dev_ws/yolo_tut/datasets/people_walk.mp4: 384x640 7 persons, 7.3ms
video 1/1 (frame 5/341) /home/roni/dev_ws/yolo_

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: ultralytics.engine.results.Keypoints object
 masks: None
 names: {0: 'person'}
 obb: None
 orig_img: array([[[129, 129, 129],
         [129, 129, 129],
         [129, 129, 129],
         ...,
         [147, 150, 148],
         [148, 151, 149],
         [149, 152, 150]],
 
        [[129, 129, 129],
         [129, 129, 129],
         [129, 129, 129],
         ...,
         [147, 150, 148],
         [147, 150, 148],
         [149, 152, 150]],
 
        [[129, 129, 129],
         [129, 129, 129],
         [129, 129, 129],
         ...,
         [146, 149, 147],
         [147, 150, 148],
         [148, 151, 149]],
 
        ...,
 
        [[199, 202, 200],
         [199, 202, 200],
         [199, 202, 200],
         ...,
         [229, 229, 229],
         [229, 229, 229],
         [229, 229, 229]],
 
        [[199, 202, 200],
         [199, 202, 200],
         [199, 202,

In [ ]:
source_path_01 = "/home/roni/dev_ws/yolo_tut/datasets/le-sserafim-reuserapim-crazy-official-mv_video_720p_original.mp4"

model = YOLO("yolov8n-pose.pt")

model(source = source_path_01, show = True,  imgsz = 640)


WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/4075) /home/roni/dev_ws/yolo_tut/datasets/le-sserafim-reuserapim-crazy-official-mv_video_720p_original.mp4: 384x640 16 persons, 6.9ms
video 1/1 (frame 2/4075) /home/roni/dev_ws/yolo_tut/datasets/le-sserafim-reuserapim-crazy-official-mv_video_720p_original.mp4: 384x640 16 persons, 5.6ms
video 1/1 (frame 3/4075) /home/roni/dev_ws/yolo_tut/datasets/le-sserafim-reuserapim-crazy-official-mv_video_720p_original.mp4: 384x640 17 persons, 5.5

In [2]:
from ultralytics import YOLO
import cv2

import numpy as np

In [4]:
model = YOLO("yolov8n-pose.pt")

cap = cv2.VideoCapture(0)



while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    frame = cv2.resize(frame, (1280, 720))
    results = model(frame, stream = True)

    for r in results:
        keypoints = r.keypoints

        for ks in keypoints:
            k = np.array(ks.xy[0], dtype = int)

            for org in k:
                cv2.line(frame, org, org, (0, 0, 255), 10)

    
    cv2.imshow("Result Preview", frame)

    
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

[ WARN:0@1277.286] global cap_v4l.cpp:914 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@1277.287] global obsensor_uvc_stream_channel.cpp:163 getStreamChannelGroup Camera index out of range


In [5]:
model = YOLO("yolov8n-pose.pt")

cap = cv2.VideoCapture(0)



while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    frame = cv2.resize(frame, (1280, 720))
    results = model(frame, stream = True)

    for r in results:
        keypoints = r.keypoints

        for ks in keypoints:
            k = np.array(ks.xy[0], dtype = int)



            try:
                org = k[3]
                cv2.line(frame, org, org, (0, 0, 255), 10)

                org = k[4]
                cv2.line(frame, org, org, (255, 0, 0), 10)


            except:
                continue
    
    cv2.imshow("Result Preview", frame)

    
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


0: 384x640 1 person, 8.0ms


/tmp/ipykernel_8111/1964933505.py:20: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  k = np.array(ks.xy[0], dtype = int)


TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [1]:
from ultralytics import YOLO
import cv2

import numpy as np

In [4]:
model = YOLO('yolov8n-pose.pt')

source_path = "/home/roni/dev_ws/yolo_tut/datasets/le-sserafim-reuserapim-crazy-official-mv_video_720p_original.mp4"
cap = cv2.VideoCapture(source_path)


w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH,
                                        cv2.CAP_PROP_FRAME_HEIGHT,
                                        cv2.CAP_PROP_FPS))

img2 = cv2.imread('/home/roni/dev_ws/yolo_tut/datasets/Santa-Claus-Hat-transparent-png.png')
img2_resized = cv2.resize(img2, (80, 80))
img2_h, img2_w, img2_c = img2_resized.shape


while True:
    ret, frame = cap.read()

    if not ret:
        print("video frame is empty or video processing has been successfully completed")
        break

    
    frame = cv2.resize(frame, (1280, 720))
    results = model(frame, stream = True)


    for r in results:
        keypoints = r.keypoints
        # keypoints를 가져오는 함수 
        # 데이터 형태는 ultralytics.engine.results.Keypoints object with attributes:
        # 객체의 형태로 받아옴
        # . 텐서의 모양 (Shape): (N, K, 2)
        # N : 사람, K : 키포인트 2: X좌표 Y좌표
        for kp in keypoints:

            k = np.array(kp.xy[0], dtype = np.int32)
            # k = 각 사람들의 코 좌표를 numpy 배열로 변환
            

            try:
                org = k[0]

                x_offset = org[0] - int(img2_w // 2)
                # 코의 x 좌표 - 이미지 폭의 절반
                y_offset = org[1] - int(img2_h // 2)
                # 코의 y 좌표 - 이미지 높이의 절반


                roi = frame[y_offset : y_offset +img2_h,
                            x_offset : x_offset + img2_w]
                
                # Region of Interest, 관심 영역

                # y_offset : y_offset + img2_h
                # y_offset지점부터 이미지의 높이 만큼 더한 지점까지의 영억
                # x_offset : x_offset + img2_w
                # x_offset지점부터 이미지의 폭 만큼 더한 지점까지의 영억


                img2_gray = cv2.cvtColor(img2_resized, cv2.COLOR_BGR2GRAY)
                # 마스크를 만들기 위한 준비 BGR 이미지를 grayscale로 변환

                mask_inv = cv2.bitwise_not(img2_gray)
                # 이미지 반전 white부분을 검정색으로 바꾼다



                fg = cv2.bitwise_or(img2, img2,  mask = mask_inv)
                # foreGround(전경) 이미지
                # mask_inv의 검정색 부분은 무시 되고 흰 부분 영역만 표시 됨

                final_roi = cv2.bitwise_or(roi, fg)

                frame[y_offset : y_offset +img2_h,
                    x_offset : x_offset + img2_w] = final_roi

            except:
                print(e)
                continue


    cv2.imshow("result_preview", frame)

    if cv2.waitKey(1) == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()


0: 384x640 16 persons, 7.5ms


/tmp/ipykernel_14217/1708592255.py:37: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  k = np.array(kp.xy[0], dtype = np.int32)


TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.